<a href="https://colab.research.google.com/github/senjoyee/udemy_langchain_python/blob/main/langchain_python_bootcamp_4_39_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain --quiet
!pip install -qU langchain-openai --quiet

In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [5]:
import chardet

In [6]:
with open('spanish_customer_email.txt', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

In [7]:
encoding

'ISO-8859-1'

In [8]:
with open('spanish_customer_email.txt', encoding='ISO-8859-1') as file:
        spanish_email = file.read()

In [ ]:
print(spanish_email)

In [14]:
llm=ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

In [42]:
def translate_and_summarize(email):
    prompt1 = ChatPromptTemplate.from_messages([("system", "Identify the language in the text provided. Only return the language"),("user", "{email}")])
    prompt2 = ChatPromptTemplate.from_messages([("system", "Translate the given text from {language} to english"),("user", "{email}")])
    prompt3 = ChatPromptTemplate.from_messages([("system", "Your job is to summarize the text provided"),("user", "{translated_text}")])

    chain1 = LLMChain(llm=llm,prompt=prompt1,output_key="language")
    chain2 = LLMChain(llm=llm,prompt=prompt2,output_key="translated_text")
    chain3 = LLMChain(llm=llm,prompt=prompt3,output_key="summary")

    seq_chain = SequentialChain(chains=[chain1,chain2,chain3],input_variables=["email"],output_variables=["language","translated_text","summary"],verbose=True)
    result = seq_chain(email)
    return result

In [43]:
result = translate_and_summarize(spanish_email)



> Entering new SequentialChain chain...

> Finished chain.


In [44]:
result.keys()

dict_keys(['email', 'language', 'translated_text', 'summary'])

In [45]:
result["language"]

'Spanish'

In [46]:
result["translated_text"]

"Subject: Technical Issues Report - SAAS Panel Functionality\n\nDear Customer Support Team,\n\nI hope this message finds you well. I am writing to inform you about a technical issue I have encountered while using your SAAS panel product. As a loyal customer, I appreciate the value your product brings to my business, but I am currently facing a challenge that requires your expertise.\n\nI would like to describe in detail the problem I am experiencing:\n\n1. Graphic Issue: Upon logging into the SAAS panel, I have noticed that the graphs and tables on the main panel page are not rendering correctly. Data points appear distorted, and some elements overlap, making it difficult to accurately interpret the information.\n\n2. Export Function Failure: Additionally, I have been unable to export reports and data from the panel. Every time I try to export a report in CSV or PDF format, I receive an error message indicating that the export has failed. This functionality is crucial for sharing infor

In [47]:
result["summary"]

"The customer, Maria Rodriguez, is reporting technical issues with a SAAS panel product to the customer support team. The issues include graphic rendering problems, export function failures, and slow loading speeds. Despite troubleshooting by clearing cache and testing on different browsers, the problems persist. Maria requests assistance in resolving these issues promptly to improve the product's functionality for her business."